# Notebook 03 — Wikipedia Circuits Exploration

## 1. Présentation de la source : Wikipedia

Wikipedia est utilisée comme source de **référencement géographique** des circuits de Formule 1.
Cette source permet de récupérer :
- le nom du circuit,
- la localisation (ville / pays),
- les coordonnées géographiques (latitude / longitude),
- et une traçabilité via l’URL de la page.

Ces données servent principalement à :
- contextualiser les données Meteostat (sélection de stations météo pertinentes),
- enrichir et relier les données OpenF1 via une référence spatiale commune.

### Positionnement dans le pipeline ETL

Ce notebook couvre exclusivement l’étape **Extract** du pipeline ETL :

- **Extract** : scraping et structuration des données circuits (sans correction manuelle),
- **Transform** (à venir) : harmonisation et mapping avec OpenF1, gestion des cas ambigus,
- **Load** (à venir) : chargement en base PostgreSQL locale via Airflow.

### Limites et posture critique

- Source collaborative : mises à jour non garanties, structure variable selon les pages.
- Risques d’incohérences de format, de champs manquants ou de pages non standardisées.

Ces limites sont documentées et acceptées car l’usage se limite à des données
de référence relativement stables (localisation et coordonnées).

---------

## 2. Méthodologie de scraping

Objectif : extraire de manière reproductible et tracée les informations géographiques des circuits,
en respectant les principes de l’étape **Extract** (pas de correction métier, pas de jointure).

Stratégie :
1. **Page de référence** : identification des circuits via la page liste.
2. **Pages individuelles** : extraction des coordonnées depuis l’infobox (quand disponibles).

Principes retenus (Extract) :
- données conservées telles qu’observées,
- valeurs manquantes acceptées et documentées,
- traçabilité via l’URL source et la date de scraping,
- gestion des erreurs (pages indisponibles / infobox incomplètes) sans correction manuelle.

-------------

## 3. Scrapping

### Imports

In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
import time
from pathlib import Path
from datetime import datetime, timezone

### Initialisation

In [2]:
HEADERS = {
    "User-Agent": "F1PredictiveAssistant/1.0 (Academic project, data exploration)"
}

In [3]:
WIKIPEDIA_CIRCUITS_URL = "https://en.wikipedia.org/wiki/List_of_Formula_One_circuits"

In [4]:
response = requests.get(WIKIPEDIA_CIRCUITS_URL, headers=HEADERS)
response.raise_for_status()

soup = BeautifulSoup(response.text, "html.parser")

In [5]:
soup.title.text

'List of Formula One circuits - Wikipedia'

### Extraction de la liste des circuits

La page Wikipedia contient généralement plusieurs tables `wikitable` (légendes, tables historiques, etc.).
La table d’intérêt est celle listant les circuits de Formule 1.

Les données extraites à ce stade :
- nom du circuit,
- lien Wikipedia (traçabilité),
- ville / localité,
- pays.

Les coordonnées (latitude / longitude) ne sont pas dans la table principale et seront récupérées
sur les pages individuelles.

In [7]:
tables = soup.find_all("table", {"class": "wikitable"})
print(f"Nombre de tables trouvées : {len(tables)}")

Nombre de tables trouvées : 2


In [8]:
circuits_table = tables[1]

rows = circuits_table.find("tbody").find_all("tr")[1:]

print(f"Nombre de lignes (circuits) trouvées : {len(rows)}")

Nombre de lignes (circuits) trouvées : 78


In [9]:
circuits_list = []

BASE_WIKI_URL = "https://en.wikipedia.org"

for row in rows:
    cols = row.find_all("td")
    if len(cols) < 6:
        continue

    circuit_link = cols[0].find("a")
    circuit_name = circuit_link.get_text(strip=True) if circuit_link else None
    circuit_url = BASE_WIKI_URL + circuit_link["href"] if circuit_link else None

    location_links = cols[4].find_all("a")
    locality = location_links[0].get_text(strip=True) if location_links else None

    country_links = cols[5].find_all("a")
    country = country_links[-1].get_text(strip=True) if country_links else None

    circuits_list.append({
        "circuit_name": circuit_name,
        "circuit_url": circuit_url,
        "locality": locality,
        "country": country
    })

print(f"Circuits extraits depuis la liste : {len(circuits_list)}")
circuits_list[:3]

Circuits extraits depuis la liste : 78


[{'circuit_name': 'Adelaide Street Circuit',
  'circuit_url': 'https://en.wikipedia.org/wiki/Adelaide_Street_Circuit',
  'locality': 'Adelaide',
  'country': 'Australia'},
 {'circuit_name': 'Ain-Diab Circuit',
  'circuit_url': 'https://en.wikipedia.org/wiki/Ain-Diab_Circuit',
  'locality': 'Casablanca',
  'country': 'Morocco'},
 {'circuit_name': 'Aintree Motor Racing Circuit',
  'circuit_url': 'https://en.wikipedia.org/wiki/Aintree_Motor_Racing_Circuit',
  'locality': 'Aintree',
  'country': 'United Kingdom'}]

### Structuration tabulaire

Les données extraites sont converties en **DataFrame pandas** afin de faciliter :
- l’enrichissement des coordonnées,
- l’analyse de qualité,
- et la persistance des résultats de l’étape Extract.

Aucune correction métier n’est réalisée.

In [10]:
circuits_df = pd.DataFrame(circuits_list)

circuits_df

,circuit_name,circuit_url,locality,country
0,Adelaide Street Circuit,https://en.wikipedia.org/wiki/Adelaide_Street_...,Adelaide,Australia
1,Ain-Diab Circuit,https://en.wikipedia.org/wiki/Ain-Diab_Circuit,Casablanca,Morocco
2,Aintree Motor Racing Circuit,https://en.wikipedia.org/wiki/Aintree_Motor_Ra...,Aintree,United Kingdom
3,Albert Park Circuit,https://en.wikipedia.org/wiki/Albert_Park_Circuit,Melbourne,Australia
4,Algarve International Circuit,https://en.wikipedia.org/wiki/Algarve_Internat...,Portimão,Portugal
...,...,...,...,...
73,TI Circuit Aida,https://en.wikipedia.org/wiki/Okayama_Internat...,Mimasaka,Japan
74,Valencia Street Circuit,https://en.wikipedia.org/wiki/Valencia_Street_...,Valencia,Spain
75,Watkins Glen International,https://en.wikipedia.org/wiki/Watkins_Glen_Int...,Watkins Glen,United States
76,Yas Marina Circuit,https://en.wikipedia.org/wiki/Yas_Marina_Circuit,Abu Dhabi,United Arab Emirates


### Enrichissement géographique (latitude / longitude)

La latitude et la longitude ne figurent pas dans la table principale.
Chaque page circuit est visitée pour récupérer les coordonnées.

Principe Extract :
- on privilégie les coordonnées **décimales** lorsqu’elles sont disponibles,
- si les coordonnées ne sont pas présentes ou non parsables, on conserve `None`,
- aucune correction manuelle n’est appliquée.

In [11]:
def extract_coordinates(url: str, headers: dict):
    """
    Extrait les coordonnées (latitude, longitude) depuis une page Wikipedia.

    Retour :
        (lat, lon) : tuple(float|None, float|None)
    """
    try:
        response = requests.get(url, headers=headers, timeout=10)
        response.raise_for_status()
    except requests.RequestException:
        return None, None

    soup = BeautifulSoup(response.text, "html.parser")

    geo = soup.select_one("span.geo")
    if geo and ";" in geo.get_text(strip=True):
        try:
            lat_str, lon_str = geo.get_text(strip=True).split(";")
            return float(lat_str.strip()), float(lon_str.strip())
        except ValueError:
            return None, None

    return None, None

In [12]:
latitudes = []
longitudes = []

for _, row in circuits_df.iterrows():
    url = row["circuit_url"]
    lat, lon = extract_coordinates(url, HEADERS)
    latitudes.append(lat)
    longitudes.append(lon)
    time.sleep(1)

In [13]:
circuits_df["latitude"] = latitudes
circuits_df["longitude"] = longitudes

In [15]:
circuits_df.head(10)

,circuit_name,circuit_url,locality,country,latitude,longitude
0,Adelaide Street Circuit,https://en.wikipedia.org/wiki/Adelaide_Street_...,Adelaide,Australia,-34.93056,138.62056
1,Ain-Diab Circuit,https://en.wikipedia.org/wiki/Ain-Diab_Circuit,Casablanca,Morocco,33.57861,-7.68750
2,Aintree Motor Racing Circuit,https://en.wikipedia.org/wiki/Aintree_Motor_Ra...,Aintree,United Kingdom,53.47694,-2.94056
3,Albert Park Circuit,https://en.wikipedia.org/wiki/Albert_Park_Circuit,Melbourne,Australia,-37.84972,144.96833
4,Algarve International Circuit,https://en.wikipedia.org/wiki/Algarve_Internat...,Portimão,Portugal,37.23194,-8.63194
5,Autódromo do Estoril,https://en.wikipedia.org/wiki/Aut%C3%B3dromo_d...,Estoril,Portugal,38.75083,-9.39417
6,Autódromo Hermanos Rodríguez,https://en.wikipedia.org/wiki/Aut%C3%B3dromo_H...,Mexico City,Mexico,19.40611,-99.09250
7,Autódromo Internacional do Rio de Janeiro,https://en.wikipedia.org/wiki/Aut%C3%B3dromo_I...,Rio de Janeiro,Brazil,-22.97556,-43.39500
8,Autodromo Internazionale del Mugello,https://en.wikipedia.org/wiki/Autodromo_Intern...,Scarperia e San Piero,Italy,43.99750,11.37194
9,Autodromo Internazionale Enzo e Dino Ferrari,https://en.wikipedia.org/wiki/Imola_Circuit,Imola,Italy,44.34111,11.71333


----------------

## 4. Vérification et validation de la table circuits

La table `circuits_df` contient pour chaque circuit :
- nom,
- URL Wikipedia (traçabilité),
- ville et pays,
- latitude et longitude.

Avant toute transformation, une validation minimale est effectuée :
- structure (types et colonnes),
- valeurs manquantes,
- contrôles de cohérence sur les coordonnées,
- détection de doublons potentiels.

Aucune correction ou enrichissement métier n’est appliqué à ce stade.

### Aperçu général

In [16]:
circuits_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 78 entries, 0 to 77
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   circuit_name  78 non-null     object 
 1   circuit_url   78 non-null     object 
 2   locality      78 non-null     object 
 3   country       78 non-null     object 
 4   latitude      77 non-null     float64
 5   longitude     77 non-null     float64
dtypes: float64(2), object(4)
memory usage: 3.8+ KB


### Rôle des coordonnées géographiques

Latitude et longitude servent à l’alignement futur avec les données Meteostat (sélection de stations et association spatiale).

À l’étape Extract :
- les valeurs manquantes sont **acceptées**,
- elles doivent être **documentées**,
- les décisions de traitement (exclusion / enrichissement / fallback) sont reportées à Transform.

In [17]:
circuits_df.isna().sum().sort_values(ascending=False)

latitude        1
longitude       1
circuit_url     0
circuit_name    0
country         0
locality        0
dtype: int64

In [18]:
circuits_df[circuits_df["latitude"].isna() | circuits_df["longitude"].isna()]

,circuit_name,circuit_url,locality,country,latitude,longitude
19,Caesars Palace Grand Prix Circuit,https://en.wikipedia.org/wiki/Caesars_Palace_G...,Paradise,United States,NaN,NaN


Nous observons qu’il manque les coordonnées pour le circuit de Caesars Palace.
Après vérification, la page Wikipedia ne renseigne pas ces champs : il ne s’agit donc pas d’un défaut de scraping.

Ce circuit fera l’objet d’une décision lors des transformations :
- exclusion,
- enrichissement manuel documenté,
- ou fallback géographique (ex. coordonnées de la ville / zone).

--------------

## 5. Rôle des données circuits

Les circuits ne sont pas des features prédictives directes au niveau lap-level.
Ils constituent principalement une **référence spatiale** pour :
- contextualiser et aligner la météo,
- harmoniser les référentiels entre sources,
- garantir la cohérence géographique des jointures futures.

### Latitude et longitude : variables structurelles

Ces variables ne sont pas destinées à être utilisées comme features explicatives directes.
Elles permettent principalement :
- d’ancrer chaque circuit dans l’espace,
- de préparer l’alignement avec Meteostat,
- d’assurer la traçabilité géographique.

---------

## 6. Décisions ETL

Décisions appliquées dans ce notebook (Extract) :
- scraping exclusivement depuis Wikipedia,
- conservation des valeurs telles qu’observées,
- absence de correction manuelle,
- acceptation et documentation des valeurs manquantes,
- traçabilité via l’URL source et la date de scraping.

Ce notebook ne réalise :
- aucune jointure avec OpenF1 ou Meteostat,
- aucun feature engineering,
- aucune règle métier de filtrage.

--------------

## 7. Sauvegarde de la table circuits

La table circuits est sauvegardée afin de :
- figer l’état des données Wikipedia au moment du scraping,
- garantir la reproductibilité,
- éviter le re-scraping,
- faciliter la réutilisation lors des transformations futures.

Format : CSV (tabulaire, lisible, sans index).
Aucune transformation métier n’est appliquée avant export.

In [19]:
EXTRACT_DIR = Path("../data/extract/wikipedia")
EXTRACT_DIR.mkdir(parents=True, exist_ok=True)

output_path = EXTRACT_DIR / "circuits_wikipedia_extract.csv"

circuits_df = circuits_df.copy()
circuits_df["scraped_at_utc"] = datetime.now(timezone.utc).isoformat()

circuits_df.to_csv(output_path, index=False)

print(f"Table circuits sauvegardée (Extract) : {output_path}")

Table circuits sauvegardée (Extract) : ..\data\extract\wikipedia\circuits_wikipedia_extract.csv


------------

## 8. Conclusion — Wikipedia Circuits Exploration

Ce notebook a exploré et structuré les informations sur les circuits de F1 à partir de Wikipedia.

Étapes réalisées :
- identification des circuits via la page de référence,
- extraction des informations structurantes (nom, pays, localité),
- enrichissement géographique depuis les pages individuelles (quand disponible),
- construction d’une table de référence circuits,
- évaluation de la qualité et des limites de la source.

Le travail est strictement **exploratoire et préparatoire** : il produit un artefact **Extract**
réutilisable sans interaction avec les autres sources.

Les étapes suivantes (hors périmètre de ce notebook) porteront sur :
- l’alignement spatial avec les données Meteostat,
- la gestion des cas ambigus ou incomplets,
- la préparation des référentiels nécessaires aux jointures avec OpenF1.